# 5, Error back propagation method 

### 5-1 Write calcuration graph


#### Practice calcuration graph

1. 太郎君はスーパーで1個100円のリンゴを2個買いました。\
支払う金額を求めなさい。ただし、消費税10％を込みで計算すること

A. ![image1](./calcuration_graph_image/IMG_2202.png)


1. 太郎君はスーパーでリンゴを2個ミカンを3個買いました。 \
リンゴは1個100円、ミカンは1個150円です。消費税10％かかるものとして、支払う金額を求めなさい

A.![image2](./calcuration_graph_image/IMG_2203.png)

This is a forward propagation

### 5-2 Chain rule

#### 5-2-1 Composite function
$$
z = t^2 \\
t = x + y
$$

If That function was shown comosite function, it is represented by the product of the derivatives of each of the functions that make up the composite function

$$
\frac{\sigma z}{\sigma x} = \frac{\sigma z}{\sigma t}  \frac{\sigma t}{\sigma x} \\
$$

$$
=\frac{\sigma z}{\sigma x}
$$

Calculate $\frac{\sigma z}{\sigma x}$. However You must calculate partitial differential before

$$
\frac{\sigma z}{\sigma t} = 2t
$$

$$
\frac{\sigma t}{\sigma x} = 1
$$

Therefore,
$$
\frac{\sigma z}{\sigma x} = \frac{\sigma z}{\sigma t} \frac{\sigma t}{\sigma x} = 2t * 1 = 2(x + y)
$$

Calculate graph
![image3](./calcuration_graph_image/IMG_2204.jpg)


#### 5-2-2, How to calculate of  any node

##### 1. Addition node
- return same inputed value to next node

##### 2. Multiplication node
- Returns the value that is the flipped value of the input value at the time of forward propagation

if you execute this one, you must save the input value

### 5-3 Make a layer

#### 5-3-1, Multilayer


In [2]:
# forward(): Forward propagation
# backward(): Backward propagation

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return int(out)

    def backward(self, dout):
        dx = dout * self.y # flipped x , y
        dy = dout * self.x
        
        return dx, dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# debug
print(int(price))

# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)

dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax) 

220
2.2 110.00000000000001 200


#### 5-3-2 Addition layer

In [4]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y

        return int(out)

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

print(price)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(int(dapple_num), dapple, round(dorange, 1), dorange_num, dtax)

715
110 2.2 3.3 165.0 650


### 6, Activation function layer

#### 6-1,ReLU layer

- forward ver
$$
y = \begin{cases}
x & {(x > 0)} \\
0 & {(x \geqq 0)}
\end{cases}
$$

- backward ver
$$
\frac{\sigma y}{\sigma x} = \begin{cases}
1 & (x > 0) \\
0 & (x \geqq 0)
\end{cases}
$$

In [6]:
# mask: A list of Numpy as True or False
import numpy as np

x = np.array([[1.0, -0.5],[-2.0, 3.0]])
# print(x)
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


#### 6-2 sigmoid layer

- forward ver
$$
\frac{1}{1 + exp(-x)}
$$

- backward ver

### step1

"/" node is shwon $ y = \frac{1}{x}$ .
$$
\frac{\sigma y}{\sigma x} = -\frac{1}{x^2} \
 = -y^2
$$

### step2
This is a "+" node. Therefore, returns the value obtained in step1 as it is

### step3

"exp" node is shown $ y = exp(x)$. So, in this gradient is 
$$
\frac{\sigma y}{\sigma x} = exp(x)
$$

### step4
"x" node is invert and multiply the value at that time of forward propagation. \
Therefore, The result is following value

$$
\frac{\sigma L}{\sigma y} y^2 exp(-x)
$$

Thus, the formula obtained above can be organaized as follows
$$
\begin{align}
\frac{\sigma L}{\sigma y} y^2 exp(-x) &= \frac{\sigma L}{\sigma y}\frac{1}{(1 + exp(-x))^2}exp(-x) \\
&= \frac{\sigma L}{\sigma y}\frac{1}{(1 + exp(-x))}\frac{exp(-x)}{1 + exp(-x)} \\
&= \frac{\sigma L}{\sigma y} y(1-y)
\end{align}
$$

#### 6-3 Affine layer

In this sentence, Backpropagation of the matrix will be performed

$$
\frac{\sigma L}{\sigma X} = \frac{\sigma L}{\sigma Y} * W^T \\
\frac{\sigma L}{\sigma W} = X^T * \frac{\sigma L}{\sigma Y}
$$

calculation graph
![image4](./calcuration_graph_image/IMG_2207.jpg)

#### 6-3-1 Affine layer for batch version

Basically the same but, be careful when adding bias

In [8]:
# forward example
x_dot_w = np.array([[0, 0, 0], [10, 10, 10]])

B = np.array([1, 2, 3])

print(x_dot_w)

print(x_dot_w + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


In [11]:
# backward example
dy = np.array([[1, 2, 3],[4, 5, 6]])
print(dy)

db = np.sum(dy, axis=0)
print(db)

[[1 2 3]
 [4 5 6]]
[5 7 9]


#### 6-4 Softmax-with-Loss layer

It is usually using output layer

example: Mnist data
![image5](./calcuration_graph_image/IMG_2209.jpg)

### 7 Execute back propagation error method

In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from shared_code.layer import *
from shared_code.gradient import numerical_gradient
from collections import OrderedDict

In [ ]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std=0.01):
        
        # Weight initialization
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # Make a layer
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # x: input data  t:training data
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)

        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)

        return grads